# SBTi-Finance Tool - Portfolio Aggregation
In this notebook we'll give some examples on how the portfolio aggregation methods can be used.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf), [guidance](https://sciencebasedtargets.org/financial-institutions) and the [technical documentation](http://getting-started.sbti-tool.org/)  for more details on the different aggregation methods.

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 


## Setting up
First we will set up the imports, data providers, and load the portfolio. 

For more examples of this process, please refer to notebook 1 & 2 (analysis and quick calculation example).


In [ ]:
!pip install SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")


In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
scores_collection = {}

In [4]:
temperature_score = TemperatureScore(time_frames=list(SBTi.interfaces.ETimeFrames), scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3])
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This method uses the "investment_value" field to be defined in your portfolio data.

In [5]:
temperature_score.aggregation_method = PortfolioAggregationMethod.WATS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_wats = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'WATS': df_wats})
df_wats

,short,mid,long
S1S2,2.54,2.63,3.16
S3,2.93,2.98,3.23
S1S2S3,2.83,2.88,3.21


### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 
In addition to the portfolios "investment value" the TETS method requires company emissions, please refer to [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [6]:
temperature_score.aggregation_method = PortfolioAggregationMethod.TETS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_tets = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'TETS': df_tets})
df_tets

,short,mid,long
S1S2,2.84,2.92,2.93
S3,2.50,2.76,3.21
S1S2S3,2.62,2.82,3.11


### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.
In addition to the portfolios "investment value" the MOTS method requires company emissions and market cap, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [7]:
temperature_score.aggregation_method = PortfolioAggregationMethod.MOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_mots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'MOTS': df_mots})
df_mots

,short,mid,long
S1S2,2.95,3.04,3.13
S3,2.66,2.47,3.20
S1S2S3,2.76,2.67,3.18


### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach. 
In addition to the portfolios "investment value" the EOTS method requires company emissions and enterprise value, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [8]:
temperature_score.aggregation_method = PortfolioAggregationMethod.EOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_eots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'EOTS': df_eots})
df_eots

,short,mid,long
S1S2,2.99,3.02,3.16
S3,2.50,2.20,3.20
S1S2S3,2.69,2.52,3.19


### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach. 
In addition to the portfolios "investment value" the ECOTS method requires company emissions, company cash equivalents and enterprise value; please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [9]:
temperature_score.aggregation_method = PortfolioAggregationMethod.ECOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_ecots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ECOTS': df_ecots})
df_ecots

,short,mid,long
S1S2,2.96,3.00,3.14
S3,2.51,2.41,3.21
S1S2S3,2.68,2.63,3.18


### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach. 
In addition to the portfolios "investment value" the AOTS method requires company emissions and company total assets; please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [10]:
temperature_score.aggregation_method = PortfolioAggregationMethod.AOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_aots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'AOTS': df_aots})
df_aots

,short,mid,long
S1S2,2.01,2.09,3.19
S3,0.87,3.11,3.21
S1S2S3,1.22,2.80,3.21


### ROTS
Revenue owned emissions weighted temperature score (ROTS): Temperature scores are allocated based on the share of revenue.
In addition to the portfolios "investment value" the ROTS method requires company emissions and company revenue; please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [11]:
temperature_score.aggregation_method = PortfolioAggregationMethod.ROTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_rots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ROTS': df_rots})
df_rots

,short,mid,long
S1S2,3.06,3.06,3.10
S3,2.68,2.47,3.20
S1S2S3,2.83,2.70,3.16


See below how each aggregation method impact the scores on for each time frame and scope combination

In [12]:
pd.concat(scores_collection, axis=0)

short   mid  long
WATS  S1S2     2.54  2.63  3.16
      S3       2.93  2.98  3.23
      S1S2S3   2.83  2.88  3.21
TETS  S1S2     2.84  2.92  2.93
      S3       2.50  2.76  3.21
      S1S2S3   2.62  2.82  3.11
MOTS  S1S2     2.95  3.04  3.13
      S3       2.66  2.47  3.20
      S1S2S3   2.76  2.67  3.18
EOTS  S1S2     2.99  3.02  3.16
      S3       2.50  2.20  3.20
      S1S2S3   2.69  2.52  3.19
ECOTS S1S2     2.96  3.00  3.14
      S3       2.51  2.41  3.21
      S1S2S3   2.68  2.63  3.18
AOTS  S1S2     2.01  2.09  3.19
      S3       0.87  3.11  3.21
      S1S2S3   1.22  2.80  3.21
ROTS  S1S2     3.06  3.06  3.10
      S3       2.68  2.47  3.20
      S1S2S3   2.83  2.70  3.16